In [2]:
import pandas as pd
import numpy as np
from CloseEncountersH3 import *
from CloseEncountersBF import *

spark = SparkSession.builder \
    .appName("CloseEncountersH3") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.rpc.message.maxSize", 1028) \
    .getOrCreate()

coords_df = pd.read_parquet('~/Repos/close-encounters/data/flight_profiles_cpf_20240701_filtered.parquet')
f = np.logical_and(coords_df.TIME_OVER >= datetime(2024,7,1,12,5,0), coords_df.TIME_OVER<= datetime(2024,7,1,12,10,0))
coords_df = coords_df[f]

ce_h3_df = CloseEncountersH3(coords_df, distance_nm = 5, FL_diff = 9, FL_min = 250, deltaT_min = 10, spark = spark)

#ce_bf_df = CloseEncountersBF(coords_df, distance_nm = 5, FL_diff = 9, FL_min = 250, deltaT_min = 10, spark = spark)

The selected resolution for a distance of 5 NM is: 5
Number of rows as input: (24893, 5)


/Users/quintengoens/Repos/close-encounters/.conda/lib/python3.12/site-packages/tempo/utils.py:122: ResampleWarning: 
            Resample Metrics Warning:
                Earliest Timestamp: 2024-07-01 12:05:00
                Latest Timestamp: 2024-07-01 12:10:00
                No. of Unique Partitions: 2828
                Resampled Min No. Values in Single a Partition: 1.0
                Resampled Max No. Values in Single a Partition: 61.0
                Resampled P25 No. Values in Single a Partition: 52.0
                Resampled P50 No. Values in Single a Partition: 55.0
                Resampled P75 No. Values in Single a Partition: 56.0
                Resampled Total No. Values Across All Partitions: 145432.0
        
  warnings.warn(


Number of rows after resamplin and interpolating: 145432


Number of pairs (raw): 118648
Number of pairs after time filter 118648
Number of pairs after FL filter 11077


25/05/07 16:55:58 WARN CacheManager: Asked to cache already cached data.


Number of unique ID pairs: 3


In [3]:
ce_h3_df

,ID2,ID1,time_over,h3_group,ID,lat1,lon1,time1,flight_lvl1,flight_id1,lat2,lon2,time2,flight_lvl2,flight_id2,time_diff_s,FL_diff,distance_nm
0,17179908034,17179897081,2024-07-01 12:06:15,851e34cbfffffff,17179897081_17179908034,50.221343,12.152176,2024-07-01 12:06:15,369.166667,273710082.0,50.222153,12.137917,2024-07-01 12:06:15,360.250,273711122.0,0,8.916667,0.550529
1,17179908033,17179897080,2024-07-01 12:06:10,851fa9affffffff,17179897080_17179908033,50.232685,12.151574,2024-07-01 12:06:10,369.333333,273710082.0,50.219618,12.154236,2024-07-01 12:06:10,360.375,273711122.0,0,8.958333,0.792061
2,17179908035,17179897082,2024-07-01 12:06:20,851fa9affffffff,17179897082_17179908035,50.210000,12.152778,2024-07-01 12:06:20,369.000000,273710082.0,50.224687,12.121597,2024-07-01 12:06:20,360.125,273711122.0,0,8.875000,1.489129


In [5]:
ce_bf_df.toPandas()

,ID2,ID1,ID,lat1,lon1,time1,flight_lvl1,flight_id1,lat2,lon2,time2,flight_lvl2,flight_id2,time_diff_s,FL_diff,distance_nm
0,17179908033,17179897080,17179897080_17179908033,50.232685,12.151574,2024-07-01 12:06:10,369.333333,273710082.0,50.219618,12.154236,2024-07-01 12:06:10,360.375,273711122.0,0,8.958333,0.792061
1,17179908035,17179897082,17179897082_17179908035,50.210000,12.152778,2024-07-01 12:06:20,369.000000,273710082.0,50.224687,12.121597,2024-07-01 12:06:20,360.125,273711122.0,0,8.875000,1.489129
2,17179908034,17179897081,17179897081_17179908034,50.221343,12.152176,2024-07-01 12:06:15,369.166667,273710082.0,50.222153,12.137917,2024-07-01 12:06:15,360.250,273711122.0,0,8.916667,0.550529


In [ ]:
# -----------------------------------------------------------------------------
# Imports
# -----------------------------------------------------------------------------

# PySpark libraries
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import (
    udf, col, explode, radians,
    sin, cos, sqrt, atan2, lit, monotonically_increasing_id
)
from pyspark.sql.types import (
    StringType, ArrayType )
from pyspark.sql import Window
from tempo import *

# Data libraries
import h3
import pandas as pd
import numpy as np
from datetime import datetime

# Custom libraries
from helpers import select_resolution

# -----------------------------------------------------------------------------
# Spark Configuration
# -----------------------------------------------------------------------------
spark = SparkSession.builder \
    .appName("CloseEncountersH3") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.rpc.message.maxSize", 1028) \
    .getOrCreate()

# -----------------------------------------------------------------------------
# Close encounter parameters
# -----------------------------------------------------------------------------
# Minimal horizontal distance before close encounter (NM)
distance_nm = 5

# Minimal vertical distance before close encounter (flight levels - FL)
FL_diff = 9

# The minimum flight level for assessment of the trajectory (lower sections are not analyzed)
FL_min = 250

# The maximum period we should interpolate in case of missing state-vectors (deltaT in minutes)
deltaT_min = 10

# -----------------------------------------------------------------------------
# Default / Automatic parameters
# -----------------------------------------------------------------------------
def h3_encounter(coords_df, distance_nm = 5, FL_diff = 9, FL_min = 250, deltaT_min = 10, spark = None):
    resolution = select_resolution(distance_nm)
    earth_radius_km = 6378
    print(f"The selected resolution for a distance of {distance_nm} NM is: {resolution}")

    # -----------------------------------------------------------------------------
    # Load and Filter Data
    # -----------------------------------------------------------------------------
    coords_df = coords_df[coords_df.FLIGHT_LEVEL > FL_min]
    coords_df = coords_df[['FLIGHT_ID', 'LONGITUDE', 'LATITUDE', 'TIME_OVER', 'FLIGHT_LEVEL']].rename(
        columns={
            'LATITUDE': 'latitude', 
            'LONGITUDE': 'longitude'
            }
        )
    coords_df.columns = [x.lower() for x in coords_df.columns]

    print(f"Number of rows as input: {coords_df.shape}")
    coords_df = spark.createDataFrame(coords_df)

    # -----------------------------------------------------------------------------
    # Resample and interpolate
    # -----------------------------------------------------------------------------

    coords_df = TSDF(coords_df, ts_col="time_over", partition_cols = ["flight_id"])
    coords_df = coords_df.resample(freq="5 sec", func="mean").interpolate(method='linear', freq="5 sec", show_interpolated = True).df
    coords_df = coords_df.repartition(100, ["flight_id"])
    print(f"Number of rows after resamplin and interpolating: {coords_df.count()}")

    # -----------------------------------------------------------------------------
    # Delete resampled periods which are longer than DeltaT = 10 min
    # -----------------------------------------------------------------------------

    # Define a window partitioned by flight and segment and ordered by time
    w = Window.partitionBy("flight_id").orderBy("time_over")

    # Flag changes in interpolation status (start of new group)
    coords_df = coords_df.withColumn(
        "interpolation_group_change",
        (F.col("is_ts_interpolated") != F.lag("is_ts_interpolated", 1).over(w)).cast("int")
    )

    # Fill nulls in the first row with 1 (new group)
    coords_df = coords_df.withColumn(
        "interpolation_group_change",
        F.when(F.col("interpolation_group_change").isNull(), 1).otherwise(F.col("interpolation_group_change"))
    )

    # Create a cumulative sum over the changes to assign group IDs
    coords_df = coords_df.withColumn(
        "interpolation_group_id",
        F.sum("interpolation_group_change").over(w)
    )

    # Add min and max timestamp per interpolation group
    group_window = Window.partitionBy("flight_id", "interpolation_group_id")

    coords_df = coords_df.withColumn("group_start_time", F.min("time_over").over(group_window))
    coords_df = coords_df.withColumn("group_end_time", F.max("time_over").over(group_window))

    # Calculate duration in seconds for each interpolation group
    coords_df = coords_df.withColumn(
        "interpolation_group_duration_sec",
        F.col("group_end_time").cast("long") - F.col("group_start_time").cast("long")
    )

    # Filter logic:
    # - If not interpolated, keep
    # - If interpolated, keep only if group duration <= deltaT_min * 60 seconds
    coords_df = coords_df.filter(
        (~F.col("is_ts_interpolated")) |
        ((F.col("is_ts_interpolated")) & (F.col("interpolation_group_duration_sec") <= deltaT_min*60))
    )

    # Drop helper columns
    coords_df = coords_df.drop("interpolation_group_change", "interpolation_group_id",
                            "group_start_time", "group_end_time", "interpolation_group_duration_sec")

    # Add a segment ID
    coords_df = coords_df.withColumn("segment_id", monotonically_increasing_id())
    coords_df = coords_df.repartition(100, ["flight_id", "segment_id"])

    #coords_df = coords_df.filter(col('time_over')==datetime(2024,7,1,12,1,0))
    coords_df.cache()
    coords_df.count()

    # -----------------------------------------------------------------------------
    # Define UDFs for H3
    # -----------------------------------------------------------------------------
    def lat_lon_to_h3(lat, lon, resolution):
        return h3.latlng_to_cell(lat, lon, resolution)

    def grid_disk_k1(cell):
        return h3.grid_disk(cell, k=1)

    lat_lon_to_h3_udf = udf(lat_lon_to_h3, StringType())
    grid_disk_k1_udf = udf(grid_disk_k1, ArrayType(StringType()))

    # Add H3 index and neighbors
    coords_df = coords_df.withColumn("h3_index", lat_lon_to_h3_udf(col("latitude"), col("longitude"), lit(resolution)))
    coords_df = coords_df.withColumn("h3_neighbours", grid_disk_k1_udf(col("h3_index")))

    # -----------------------------------------------------------------------------
    # Explode neighbors and group by time_over and h3_neighbour to collect IDs when there's multiple FLIGHT_ID in a cell
    # -----------------------------------------------------------------------------
    exploded_df = coords_df.withColumn("h3_neighbour", explode(col("h3_neighbours")))

    grouped_df = (exploded_df.groupBy(["time_over", "h3_neighbour"])
                .agg(F.countDistinct("flight_id").alias("flight_count"),
                    F.collect_list("segment_id").alias("id_list"))
                .filter(F.col("flight_count") > 1)
                .drop("flight_count"))

    grouped_df = grouped_df.filter(F.size("id_list") > 1)

    # -----------------------------------------------------------------------------
    # Create pairwise combinations using self-join on indexed exploded DataFrame
    # -----------------------------------------------------------------------------
    # Explode id_list to individual rows and add index within each h3 group
    df_exploded = grouped_df.withColumn("segment_id", explode("id_list")).drop('id_list')
    window_spec = Window.partitionBy(["time_over","h3_neighbour"]).orderBy("segment_id")
    df_indexed = df_exploded.withColumn("idx", F.row_number().over(window_spec))

    # Self-join to form unique unordered ID pairs
    df_pairs = (
        df_indexed.alias("df1")
        .join(
            df_indexed.alias("df2"),
            (F.col("df1.time_over") == F.col("df2.time_over")) &
            (F.col("df1.h3_neighbour") == F.col("df2.h3_neighbour")) &
            (F.col("df1.idx") < F.col("df2.idx"))
        )
        .select(
            F.col("df1.time_over").alias("time_over"),
            F.col("df1.h3_neighbour").alias("h3_group"),
            F.col("df1.segment_id").alias("ID1"),
            F.col("df2.segment_id").alias("ID2")
        )
    )

    # -----------------------------------------------------------------------------
    # Clean Pairs, Create Unique Pair ID
    # -----------------------------------------------------------------------------
    df_pairs = df_pairs.filter(col("ID1") != col("ID2")) # should not be necessary as we join on < not <=
    df_pairs = df_pairs.withColumn(
        "ID",
        F.concat_ws("_", F.array_sort(F.array(col("ID1"), col("ID2"))))
    )

    # Define a window partitioned by ID, ordering arbitrarily (or by some column if needed)
    window_spec = Window.partitionBy("ID").orderBy(F.monotonically_increasing_id())

    # Add row number to each partition
    df_pairs = df_pairs.withColumn("row_num", F.row_number().over(window_spec))

    # Keep only the first row per ID
    df_pairs = df_pairs.filter(F.col("row_num") == 1).drop("row_num")

    # -----------------------------------------------------------------------------
    # Join with Original Coordinates for Each ID
    # -----------------------------------------------------------------------------
    coords_sdf1 = coords_df.withColumnRenamed("segment_id", "ID1") \
        .withColumnRenamed("latitude", "lat1") \
        .withColumnRenamed("longitude", "lon1") \
        .withColumnRenamed("time_over", "time1") \
        .withColumnRenamed("flight_level", 'flight_lvl1') \
        .withColumnRenamed("flight_id", "flight_id1") \
        .select("ID1", "lat1", "lon1", "time1", "flight_lvl1", "flight_id1")

    coords_sdf2 = coords_df.withColumnRenamed("segment_id", "ID2") \
        .withColumnRenamed("latitude", "lat2") \
        .withColumnRenamed("longitude", "lon2") \
        .withColumnRenamed("time_over", "time2") \
        .withColumnRenamed("flight_level", 'flight_lvl2') \
        .withColumnRenamed("flight_id", "flight_id2") \
        .select("ID2", "lat2", "lon2", "time2", "flight_lvl2", "flight_id2")

    coords_sdf1 = coords_sdf1.repartition(100, "ID1")
    coords_sdf2 = coords_sdf2.repartition(100, "ID2")

    df_pairs = df_pairs.join(coords_sdf1, on="ID1", how="left")
    df_pairs = df_pairs.join(coords_sdf2, on="ID2", how="left")
    df_pairs.cache()
    print(f"Number of pairs (raw): {df_pairs.count()}")
    # -----------------------------------------------------------------------------
    # Calculate and filter based on time differense (s)
    # -----------------------------------------------------------------------------
    df_pairs = df_pairs.withColumn('time_diff_s', F.unix_timestamp(F.col("time1")) - F.unix_timestamp(F.col("time2")))
    df_pairs = df_pairs.filter(F.abs(F.col('time_diff_s')) == 0)
    df_pairs.cache()
    print(f"Number of pairs after time filter {df_pairs.count()}")
    # -----------------------------------------------------------------------------
    # Calculate and filter based on height differense (s)
    # -----------------------------------------------------------------------------
    df_pairs = df_pairs.withColumn('FL_diff', F.col("flight_lvl1") - F.col("flight_lvl2"))
    df_pairs = df_pairs.filter(F.abs(F.col('FL_diff')) < lit(FL_diff))
    df_pairs.cache()
    print(f"Number of pairs after FL filter {df_pairs.count()}")

    # -----------------------------------------------------------------------------
    # Calulate and filter based on distance (km)
    # -----------------------------------------------------------------------------
    df_pairs.cache()
    df_pairs = df_pairs.withColumn(
        "distance_nm",
        0.539957 * 2 * earth_radius_km * atan2(
            sqrt(
                (sin(radians(col("lat2")) - radians(col("lat1"))) / 2)**2 +
                cos(radians(col("lat1"))) * cos(radians(col("lat2"))) *
                (sin(radians(col("lon2")) - radians(col("lon1"))) / 2)**2
            ),
            sqrt(1 - (
                (sin(radians(col("lat2")) - radians(col("lat1"))) / 2)**2 +
                cos(radians(col("lat1"))) * cos(radians(col("lat2"))) *
                (sin(radians(col("lon2")) - radians(col("lon1"))) / 2)**2
            ))
        )
    )

    df_pairs = df_pairs.filter(col('distance_nm') <= lit(distance_nm))

    # -----------------------------------------------------------------------------
    # Fetch sample
    # -----------------------------------------------------------------------------

    df_pairs.cache()
    df = df_pairs.toPandas()
    print(f"Number of unique ID pairs: {df_pairs.count()}")
    return df

In [ ]:
df

In [ ]:
import plotly.express as px
max_distance = np.sqrt(3)/2*10.837435124897937*4
fig = px.histogram(df, x='distance_nm')

fig.add_vline(x=max_distance, line_dash = 'dash', line_color = 'firebrick')

In [ ]:
coords_df = pd.read_parquet('~/Repos/close-encounters/data/flight_profiles_cpf_20240701_filtered.parquet')
coords_df['SEGMENT_ID'] = coords_df.index
coords_df = coords_df[coords_df.FLIGHT_LEVEL > 250]
coords_df = coords_df[['FLIGHT_ID', 'SEGMENT_ID', 'LONGITUDE', 'LATITUDE', 'TIME_OVER', 'FLIGHT_LEVEL', 'AIRCRAFT_TYPE']].rename(
    columns={
        'LATITUDE': 'latitude', 
        'LONGITUDE': 'longitude'
        }
    )

c1 = coords_df[['FLIGHT_ID', 'SEGMENT_ID', 'AIRCRAFT_TYPE']].rename(
    {'FLIGHT_ID':'FLIGHT_ID1', 'SEGMENT_ID':'ID1', 'AIRCRAFT_TYPE':'AC1'},axis=1)
c2 = coords_df[['FLIGHT_ID', 'SEGMENT_ID', 'AIRCRAFT_TYPE']].rename(
    {'FLIGHT_ID':'FLIGHT_ID2', 'SEGMENT_ID':'ID2', 'AIRCRAFT_TYPE':'AC2'},axis=1)

df = df.merge(c1, how='left').merge(c2, how='left')

In [ ]:
coords_df['FLIGHT_ID'] = coords_df['FLIGHT_ID'].apply(str) + '_id'
df['FLIGHT_ID1'] = df['FLIGHT_ID1'].apply(str) + '_id'
df['FLIGHT_ID2'] = df['FLIGHT_ID2'].apply(str) + '_id'
coords_df = coords_df[coords_df.FLIGHT_ID.isin(df.FLIGHT_ID1.to_list() + df.FLIGHT_ID2.to_list())]

In [ ]:
coords_df.to_parquet('coords.parquet')

In [ ]:
df.to_parquet('pairs.parquet')

In [ ]:
print(df[df.ID2 == 2557313].lon2.values[0])

In [ ]:
print(coords_df[coords_df.SEGMENT_ID == 2557313].longitude.values[0])

In [ ]:
df_pairs.toPandas()